In [1]:
# camera_calibration
'''
카메라 보정
카메라나 비디오 영상의 왜곡 현상과 카메라의 내부, 외부 파라메터에 대해 학습하며, 
이런 파라메터를 얻는 방법과 이미지의 왜곡 현상을 제거하는 내용을 설명합니다.
현대의 값 싼 소형 카메라은 상당한 이미지의 왜곡을 발생시킵니다. 
주요한 왜곡은 방사 왜곡과 탄젠티얼(Tangential) 왜곡이 있습니다.
방사 왜곡으로 인해, 아래의 그림처럼 반듯한 형상이 휘어지게 됩니다. 
이런 현상은 이미지의 중심으로부터 멀어질수록 심해집니다.
스테레오 어플리케이션에서는 이러한 왜곡을 가장 먼저 보정
'''
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('chess/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
# 동일한 카메로 위치와 각도에서, 서로 다른 체스판의 위치로 촬영된 13개의 이미지를 활용하여 패턴을 검출
cv2.destroyAllWindows()

'''
패턴 검출을 통해 객체 지점(objpoints)과 이미지 지점(imgpoints)을 파악
이를 이용해 왜곡된 촬영 영상을 보정
보정 전에 cv2.getOptimalNewCameraMatrix() 함수를 이용해 먼저 카메라 메트릭스를 구함 
이 함수는 카메라 메트릭스, 왜곡 계수, 회전/이동 벡터 등이 반환
'''
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
img = cv2.imread('chess/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
# 왜곡을 제거
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('./img/calibresult.png',dst)
'''
실행해보면, 먼저 13장의 샘플 이미지를 통해 패턴을 분석
분석된 패턴을 통해 카메라 메트릭스가 얻어지며 이 메트릭스를 개선한 뒤 최종적으로 왜곡된 부분을 제거
calibresult.png 파일로 저장
'''

tot_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist) # 왜곡 제거 시 수행된 프로젝션에 발생하는 오차 결과적으로 얻어지는 값이 0에 가까울수록 정확
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error
print("total error: ", tot_error/len(objpoints))


total error:  0.023686000375385676


In [2]:
# pose_estimation
import numpy as np
import cv2
import glob
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
objpoints = []
imgpoints = []

images = glob.glob('chess/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)
    
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
#draw라는 사용자 정의 함수를 만들텐데, 이 함수는 cv2.findChessboardCorners() 함수를 통해 
# 구한 체스판의 코너점과 축의 포인트를 인자로 받아 3차원 축을 그림
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img
# 축의 포인트를 위한 변수를 정의할 것인데, 이 변수는 축을 그리기 위한 3차원 공간 상의 포인트 
# 길이 3(단위는 체스보드의 크기를 기반으로 함)만큼을 축의 길이로 정함. 

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,  cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)
    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist) # 회전과 이동을 계산
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw(img, corners2, imgpts)
        cv2.imshow('img',img)
        k = cv2.waitKey(500)


def draw(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]),tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img
#변경된 draw 함수에 맞게 기존의 axis 변수도 큐브를 구성하는 8개의 모서리 좌표에 맞게 변경

def draw_cube(img,corners,imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)

    # 바닥 부분을 초록색으로
    img = cv2.drawContours(img,[imgpts[:4]],-1,(0,255,0),-3)

    # 기둥은 파란색으로
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img,tuple(imgpts[i]),tuple(imgpts[j]),(255,0,0),3)

    # 위의 층은 빨간색으로
    img = cv2.drawContours(img,[imgpts[4:]],-1,(0,0,255),3)

    return img

axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3]])    

for name in glob.glob('chess/*.jpg'):
    img = cv2.imread(name)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray,(7,6),None)
    # 패턴 보유시 ret = True

    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

        # rotation과 translation 벡터 찾기
        _,rvecs, tvecs, inliers = cv2.solvePnPRansac(objp,corners2,mtx,dist)

        # 3D 포인트를 이미지 평면에 투영시키자
        imgpts, jac = cv2.projectPoints(axis,rvecs,tvecs,mtx,dist)

        # 그리기
        img = draw_cube(img,corners2,imgpts)
        cv2.imshow("Res",img)
        k = cv2.waitKey(0) & 0xff
        if k == "s":
            cv2.imwrite(name[:6]+"_cube.png",img)
cv2.destroyAllWindows()                   

<ipython-input-2-5e9325739e8f>:28: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
<ipython-input-2-5e9325739e8f>:29: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
<ipython-input-2-5e9325739e8f>:30: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
